<a href="https://colab.research.google.com/github/dhirajrm/Data-Engineering-With-Pyspark/blob/git/Bank's_Transactions_Fraud_detection_EDA_%26_Insights_using_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('bank_fraud_detection').getOrCreate()

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType

schema = StructType([StructField('TransactionID', StringType(), True),
                     StructField('AccountID', StringType(), True),
                     StructField('TransactionAmount', FloatType(), True),
                     StructField('TransactionDate', TimestampType(), True),
                     StructField('IP Address', StringType(), True),
                     StructField('MerchantID', StringType(), True),
                     StructField('Channel', StringType(), True),
                     StructField('CustomerAge', StringType(), True),
                     StructField('CustomerOccupation', StringType(), True),
                     StructField('TransactionDuration', IntegerType(), True),
                     StructField('LoginAttempts', IntegerType(), True),
                     StructField('AccountBalance', FloatType(), True),
                     StructField('PreviousTransactionDate', IntegerType(), True)])

In [ ]:
bank_trans = spark.read.csv('/content/bank_transactions_data_2.csv',header = True,schema = schema)

In [ ]:
bank_trans.show()

+-------------+---------+-----------------+-------------------+----------+-------------+-------+---------------+------------------+-------------------+-------------+--------------+-----------------------+
|TransactionID|AccountID|TransactionAmount|    TransactionDate|IP Address|   MerchantID|Channel|    CustomerAge|CustomerOccupation|TransactionDuration|LoginAttempts|AccountBalance|PreviousTransactionDate|
+-------------+---------+-----------------+-------------------+----------+-------------+-------+---------------+------------------+-------------------+-------------+--------------+-----------------------+
|     TX000001|  AC00128|            14.09|2023-04-11 16:29:14|     Debit|    San Diego|D000380| 162.198.218.92|              M015|               NULL|           70|          NULL|                     81|
|     TX000002|  AC00455|           376.24|2023-06-27 16:44:19|     Debit|      Houston|D000051|    13.149.61.4|              M052|               NULL|           68|          NULL|

In [ ]:
bank_trans.printSchema()

root
 |-- TransactionID: string (nullable = true)
 |-- AccountID: string (nullable = true)
 |-- TransactionAmount: float (nullable = true)
 |-- TransactionDate: timestamp (nullable = true)
 |-- IP Address: string (nullable = true)
 |-- MerchantID: string (nullable = true)
 |-- Channel: string (nullable = true)
 |-- CustomerAge: string (nullable = true)
 |-- CustomerOccupation: string (nullable = true)
 |-- TransactionDuration: integer (nullable = true)
 |-- LoginAttempts: integer (nullable = true)
 |-- AccountBalance: float (nullable = true)
 |-- PreviousTransactionDate: integer (nullable = true)



In [ ]:
bank_trans.describe().show()

+-------+-------------+---------+------------------+----------+-----------+-------+--------------+------------------+-------------------+------------------+--------------+-----------------------+
|summary|TransactionID|AccountID| TransactionAmount|IP Address| MerchantID|Channel|   CustomerAge|CustomerOccupation|TransactionDuration|     LoginAttempts|AccountBalance|PreviousTransactionDate|
+-------+-------------+---------+------------------+----------+-----------+-------+--------------+------------------+-------------------+------------------+--------------+-----------------------+
|  count|         2512|     2512|              2512|      2512|       2512|   2512|          2512|              2512|                  0|              2512|             0|                   2512|
|   mean|         NULL|     NULL| 297.5937778157461|      NULL|       NULL|   NULL|          NULL|              NULL|               NULL|44.673964968152866|          NULL|     119.64331210191082|
| stddev|         NU

In [ ]:
bank_trans = bank_trans.dropna(how = 'all')

In [ ]:
bank_trans.describe().show()

+-------+-------------+---------+------------------+----------+-----------+-------+--------------+------------------+-------------------+------------------+--------------+-----------------------+
|summary|TransactionID|AccountID| TransactionAmount|IP Address| MerchantID|Channel|   CustomerAge|CustomerOccupation|TransactionDuration|     LoginAttempts|AccountBalance|PreviousTransactionDate|
+-------+-------------+---------+------------------+----------+-----------+-------+--------------+------------------+-------------------+------------------+--------------+-----------------------+
|  count|         2512|     2512|              2512|      2512|       2512|   2512|          2512|              2512|                  0|              2512|             0|                   2512|
|   mean|         NULL|     NULL| 297.5937778157461|      NULL|       NULL|   NULL|          NULL|              NULL|               NULL|44.673964968152866|          NULL|     119.64331210191082|
| stddev|         NU

In [ ]:
bank_trans.distinct().count()

2512

In [ ]:
# 1. Identify column types:
#    - Manually inspect the schema (bank_trans.printSchema()) and data (bank_trans.show()) to categorize columns.
bank_trans.printSchema()

root
 |-- TransactionID: string (nullable = true)
 |-- AccountID: string (nullable = true)
 |-- TransactionAmount: float (nullable = true)
 |-- TransactionDate: timestamp (nullable = true)
 |-- IP Address: string (nullable = true)
 |-- MerchantID: string (nullable = true)
 |-- Channel: string (nullable = true)
 |-- CustomerAge: string (nullable = true)
 |-- CustomerOccupation: string (nullable = true)
 |-- TransactionDuration: integer (nullable = true)
 |-- LoginAttempts: integer (nullable = true)
 |-- AccountBalance: float (nullable = true)
 |-- PreviousTransactionDate: integer (nullable = true)



In [ ]:
# 2. Create separate lists or dictionaries for each category:
Categorical_col = ['Channel','CustomerOccupation']
ID_col = ['TransactionID','AccountID','MerchantID']
Amoubt_col = ['TransactionAmount','AccountBalance']
Duration_col = ['TransactionDuration','LoginAttempts']
DateTime_col = ['TransactionDate']

In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType
from pyspark.sql.functions import when, mean, lit, col

spark = SparkSession.builder.appName('bank_fraud_detection').getOrCreate()


# Categorize columns (replace with your actual column lists):
categorical_cols = ['Channel', 'CustomerOccupation', 'IP Address']  # Added 'IP Address'
id_cols = ['TransactionID', 'AccountID', 'MerchantID']
numerical_cols = ['TransactionAmount', 'AccountBalance', 'TransactionDuration', 'LoginAttempts']
datetime_cols = ['TransactionDate', 'PreviousTransactionDate']


# Handle missing values (example – customize as needed)
for col_name in categorical_cols:
    # Impute missing categorical values with 'Unknown'
    bank_trans = bank_trans.withColumn(col_name, when(col(col_name).isNull(), 'Unknown').otherwise(col(col_name)))

for col_name in numerical_cols:
    # Calculate the mean for each numerical column
    avg_value = bank_trans.select(mean(col_name)).collect()[0][0]

    # Impute missing numerical values with the column's mean
    bank_trans = bank_trans.withColumn(col_name, when(col(col_name).isNull(), avg_value).otherwise(col(col_name)))

# Note:  'PreviousTransactionDate' is of IntegerType, so a mean will be computed as an integer

for col_name in datetime_cols:

    default_timestamp = '1900-01-01 00:00:00'  # replace with appropriate date
    bank_trans = bank_trans.withColumn(col_name, when(col(col_name).isNull(), default_timestamp).otherwise(col(col_name)))


# Verify the imputation
bank_trans.describe().show()
bank_trans.printSchema()

+-------+-------------+---------+-------------------+-------------------+----------+-----------+-------+--------------+------------------+-------------------+------------------+--------------+-----------------------+
|summary|TransactionID|AccountID|  TransactionAmount|    TransactionDate|IP Address| MerchantID|Channel|   CustomerAge|CustomerOccupation|TransactionDuration|     LoginAttempts|AccountBalance|PreviousTransactionDate|
+-------+-------------+---------+-------------------+-------------------+----------+-----------+-------+--------------+------------------+-------------------+------------------+--------------+-----------------------+
|  count|         2512|     2512|               2512|               2512|      2512|       2512|   2512|          2512|              2512|                  0|              2512|             0|                   2512|
|   mean|         NULL|     NULL|  297.5937778157461|               NULL|      NULL|       NULL|   NULL|          NULL|             